<a href="https://colab.research.google.com/github/ypsitau/pico-Recognizer-ILI9341/blob/main/pico-Recognizer-ILI9341.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Recognizer Model from MNIST

The following script mounts Google Drive and specifies the TensorFlow dataset to it. It will save time of downloading the next time you use the same dataset.

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.environ['TFDS_DATA_DIR'] = '/content/drive/MyDrive/tensorflow_datasets'
os.makedirs(os.environ['TFDS_DATA_DIR'], exist_ok=True)

Mounted at /content/drive


Running the following script creates TfLite model files. They will be automatically downloaded to your remote PC after the training.

|TfLite Model File|Recognizable Characters|
|----|----|
|`Recognizer-EMNIST-mnist-binary.tflite`|Digits (0-9)|
|`Recognizer-EMNIST-letters-binary.tflite`|Upper letters (A-Z)|
|`Recognizer-EMNIST-bymerge-binary.tflite`|Digits (0-9), upper letters (A-Z), and lower letters (a-z)|
|`Recognizer-EMNIST-balanced-binary.tflite`|Digits (0-9), upper letters (A-Z), and lower letters (a-z)|

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import google.colab

def generate_model(dataset_name: str, config_name: str | None, epochs: int, binary_image: bool) -> str:
    need_transpose = (dataset_name == 'emnist')
    label_offset = 1 if dataset_name == 'emnist' and config_name == 'letters' else 0
    dataset_fullname = dataset_name + ('/' + config_name if config_name else '')
    (dataset_train, dataset_test), dataset_info = tfds.load( # tf.data.Dataset
        dataset_fullname,
        split=['train', 'test'],
        as_supervised=True, # each data element is a tuple (image, label)
        with_info=True,
    )
    num_classes = dataset_info.features['label'].num_classes - label_offset
    #---------------------------------------------------------------------------
    print(f"{config_name}: {num_classes} classes")
    def preprocess(image: tf.Tensor, label: tf.Tensor) -> tuple[tf.Tensor, tf.Tensor]:
        if need_transpose:
            image = tf.transpose(image, perm=[1, 0, 2])
        image = tf.cast(image > 127, tf.float32) if binary_image else tf.cast(image, tf.float32) / 255.0
        label = label - label_offset
        return image, label
    dataset_train = dataset_train.map(preprocess).cache().shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
    dataset_test = dataset_test.map(preprocess).batch(64).cache().prefetch(tf.data.AUTOTUNE)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(dataset_train, epochs=epochs, validation_data=dataset_test)
    #---------------------------------------------------------------------------
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    def representative_dataset_generator():
        for image, label in dataset_test.take(100):
            yield [image]
    converter.representative_dataset = representative_dataset_generator
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    model_tflite = converter.convert()
    filename = (f"Recognizer-{dataset_name}"
        f"{'-' + config_name if config_name else ''}"
        f"{'-binary' if binary_image else ''}.tflite")
    with open(filename, "wb") as f:
        f.write(model_tflite)
    return filename

filenames = []
filenames.append(generate_model('emnist', 'mnist', 1, True))     # 0-9
#filenames.append(generate_model('emnist', 'letters', 10, True))  # A-Z, 0-9
#filenames.append(generate_model('emnist', 'bymerge', 10, True))  # 0-9, A-Z, a, b, d, e, f, g, h, n, q, r, t
#filenames.append(generate_model('emnist', 'balanced', 10, True)) # 0-9, A-Z, a, b, d, e, f, g, h, n, q, r, t
for filename in filenames:
    google.colab.files.download(filename)


mnist: 10 classes


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


437/938 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - accuracy: 0.8561 - loss: 0.4771

KeyboardInterrupt: 